## Image Retrieval with Azure AI Search

Requirements:
1. Azure AI Search
2. Azure Computer Vision
3. Azure Storage

Steps:
1. Store images in Azure Storage 
2. Get access to images in Azure Storage
3. Create Search index in Azure AI Search
4. Generate embeddings for images
5. Store embeddings and other metadata to Azure AI Search
6. Perform a search in Azure AI Search

## Load Azure Configuration

In [ ]:
import os

azure_computer_vision_endpoint = os.environ["AZURE_COMPUTER_VISION_ENDPOINT"]
azure_computer_vision_key = os.environ["AZURE_COMPUTER_VISION_KEY"]

azure_storage_connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")

azure_search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
azure_search_service_admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
azure_search_service_index_name = "az-image-index-001"

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")
azure_openai_api_version = "2024-10-01-preview"
azure_openai_embedding_size=1024


## Check Azure Storage Access

In [2]:
from azure.storage.blob import BlobServiceClient

# Initialize the BlobServiceClient with the connection string
blob_service_client = BlobServiceClient.from_connection_string(azure_storage_connection_string)

# Get the container client
container_client = blob_service_client.get_container_client("images")

# List blobs in the container
try:
    blobs_list = container_client.list_blobs()
    print("Blobs in the container:")
    for blob in blobs_list:
        print(blob.name)
    print("Access to the blob storage was granted.")
except Exception as e:
    print(f"Failed to access the blob storage: {e}")

Blobs in the container:
analyze_image.jpg
bicycle1.jpeg
bicycle10.jpeg
bicycle2.jpeg
bicycle3.jpeg
bicycle4.jpeg
bicycle5.jpeg
bicycle6.jpeg
bicycle7.jpeg
bicycle8.jpeg
bicycle9.jpeg
bus1.jpeg
bus10.jpeg
bus2.jpeg
bus3.jpeg
bus4.jpeg
bus5.jpeg
bus6.jpeg
bus7.jpeg
bus8.jpeg
bus9.jpeg
car1.jpeg
car10.jpeg
car2.jpeg
car3.jpeg
car4.jpeg
car5.jpeg
car6.jpeg
car7.jpeg
car8.jpeg
car9.jpeg
empirestate.jpeg
fork_1.jpg
fork_2.jpg
fork_3.jpg
fork_4.jpg
fork_5.jpg
group_1.jpeg
group_2.jpeg
hemlock_1.jpg
hemlock_2.jpg
hemlock_3.jpg
hemlock_4.jpg
hemlock_5.jpg
japanese_cherry_1.jpg
japanese_cherry_2.jpg
japanese_cherry_3.jpg
japanese_cherry_4.jpg
japanese_cherry_5.jpg
mtfuji.jpeg
person_a1.jpeg
person_a2.jpeg
person_b1.jpeg
person_b2.jpeg
person_c1.jpeg
person_c2.jpeg
person_c3.jpeg
scissors_1.jpg
scissors_2.jpg
scissors_3.jpg
scissors_4.jpg
scissors_5.jpg
Access to the blob storage was granted.


## Create an Azure AI Search Index

In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SimpleField,
    SearchableField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SearchIndex
)

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)

# Create a search index
index_client = SearchIndexClient(
  endpoint=azure_search_service_endpoint, 
  credential=credential)

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="image_name", type=SearchFieldDataType.String),
    SearchableField(name="image_url", type=SearchFieldDataType.String),
    SearchField(name="imageVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=azure_openai_embedding_size, vector_search_profile_name="myHnswProfile"),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(name="myHnsw")
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer_name="myVectorizer"
        )
    ],
    vectorizers=[   # a vectorizer is software that performs vectorization
        AzureOpenAIVectorizer(  
            vectorizer_name="myVectorizer",  
            kind="azureOpenAI",  
            parameters=AzureOpenAIVectorizerParameters(  
                resource_url=azure_openai_endpoint,  
                deployment_name=azure_openai_embeddings_deployment,
                model_name=azure_openai_embeddings_deployment
            ),
        ),  
    ], 
)

# Create the search index with the semantic settings
index = SearchIndex(name=azure_search_service_index_name, 
                    fields=fields,
                    vector_search=vector_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 az-image-index-001 created
